In [1]:
from data_pipeline.file_parser import process_single_file

In [ ]:
# becareful with the file name, it will be overwritten
process_single_file(
    input_pdf_path="data/raw/ftel_bctc_2024.pdf",
    output_path="data/processed/ftel_bctc_2024.md"
)

## Basic RAG

In [ ]:
import os
import yaml
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv
load_dotenv()


In [3]:
from llm_inference.llm_inference_base import get_length
from rag.indexing import create_documents, split_document, create_document
from rag.vectorstrore import initialize_vectorstore

In [ ]:
prompt_templates = yaml.safe_load(open("./prompts/rag_base.yaml"))
prompt_templates.keys()

In [5]:
config = {
    'embedding_model': 'models/text-embedding-004',
    'llm_model': 'gemini-2.5-flash',
    'chroma_db_path': './chroma_db',
    'prompt_template_path': './prompts/rag_base.yaml',
    'temperature': 0.3,
    'top_p': 0.85,
    'search_k': 2
}

In [ ]:
finacial_statement_docs = open("data/processed/ftel_bctc_2024.md", "r").read()
print(finacial_statement_docs[:1000])

In [ ]:
print(get_length(finacial_statement_docs), "tokens")

In [ ]:
## indexing
chunks = split_document(create_document(finacial_statement_docs), return_each_line=True)
print(len(chunks))

In [9]:
google_embedding = GoogleGenerativeAIEmbeddings(
    model=config["embedding_model"] 
)

In [ ]:
config["chroma_db_path"]

In [ ]:
# indexing with chroma db
vectorstore = initialize_vectorstore(
    documents=chunks,
    embeddings=google_embedding,
    db_path=config["chroma_db_path"],
    collection_name="financial_statement_01"
)

In [12]:
llm = GoogleGenerativeAI(
    model=config["llm_model"],
    temperature=config["temperature"],
    top_p=config["top_p"]
)

In [13]:
def format_docs(docs):
    try:
        context = "\n\n".join(doc.page_content for doc in docs)
    except Exception as e:
        # print(f"Error formatting documents: {e}")
        context = "\n\n".join(doc for doc in docs)
    return context

def log_prompt(prompt):
    print("\n--- Prompt Fed to LLM ---")
    try:
        print(prompt.messages[0].content)
    except:
        print(prompt)
    print("\n--- End Prompt ---")
    # print(prompt)
    return prompt

In [14]:
retriever = vectorstore.as_retriever(
    search_kwargs={"k": config["search_k"]}
)

In [ ]:
# test retriever
retriever.invoke("What is the total assets of the company?")

In [ ]:
prompt_resp = ChatPromptTemplate.from_template(prompt_templates["basic_rag_01"])
prompt_resp.pretty_print()

In [21]:
basic_rag_chain = (
    {
        "context": RunnablePassthrough() | retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt_resp
    | llm
    | StrOutputParser()
)

In [ ]:
# query
basic_rag_chain.invoke("What is the total assets of the company?")

In [ ]:
basic_rag_chain.invoke("Tính tỉ suất lợi nhuận trên vốn chủ sở hữu")

In [ ]:
print('Để tính tỉ suất lợi nhuận trên vốn chủ sở hữu (Return on Equity - ROE) cho năm 2024, chúng ta cần các thông tin sau từ ngữ cảnh:\n\n1.  **Lợi nhuận sau thuế của năm nay (Net Income for the year):** Từ Bảng Cân đối Kế toán Hợp nhất, mục 421b "Lợi nhuận sau thuế chưa phân phối của năm nay" cho năm 2024 là **2.108.378.750.719 VND**.\n2.  **Vốn chủ sở hữu (Shareholder\'s Equity):** Từ Bảng Cân đối Kế toán Hợp nhất, mục 400 "VỐN CHỦ SỞ HỮU" tại ngày 31.12.2024 là **10.646.264.236.508 VND**.\n\nCông thức tính ROE:\nROE = (Lợi nhuận sau thuế của năm nay / Vốn chủ sở hữu) * 100%\n\nThực hiện phép tính:\nROE = (2.108.378.750.719 VND / 10.646.264.236.508 VND) * 100%\nROE ≈ 0.19803 * 100%\nROE ≈ **19.80%**\n\nVậy, tỉ suất lợi nhuận trên vốn chủ sở hữu của công ty CỔ PHẦN VIỄN THÔNG FPT tại năm 2024 là khoảng **19.80%**.')

In [24]:
print(basic_rag_chain.invoke("Tính tỉ suất lợi nhuận trên vốn chủ sở hữu"))

Để tính tỉ suất lợi nhuận trên vốn chủ sở hữu (Return on Equity - ROE), ta sử dụng công thức:

**ROE = Lợi nhuận sau thuế của năm nay / Vốn chủ sở hữu**

Dựa vào ngữ cảnh:

**Năm 2024:**
*   Lợi nhuận sau thuế của năm nay (Mã số 421b): 2.108.378.750.719 VND
*   Vốn chủ sở hữu (Mã số 410): 10.646.264.236.508 VND

ROE 2024 = 2.108.378.750.719 / 10.646.264.236.508 ≈ 0.19803 hay **19.80%**

**Năm 2023:**
*   Lợi nhuận sau thuế của năm nay (Mã số 421b): 1.784.320.504.084 VND
*   Vốn chủ sở hữu (Mã số 410): 9.073.764.532.249 VND

ROE 2023 = 1.784.320.504.084 / 9.073.764.532.249 ≈ 0.19664 hay **19.66%**


In [23]:
print(basic_rag_chain.invoke("phân tích khả năng thanh toán của công ty"))

Dựa trên ngữ cảnh được cung cấp, khả năng thanh toán của công ty CỔ PHẦN VIỄN THÔNG FPT cho thấy sự cải thiện đáng kể trong năm 2024 so với năm 2023:

1.  **Tiền và các khoản tương đương tiền:**
    *   Cuối năm 2024: 679.277.313.660 VND
    *   Cuối năm 2023: 423.984.252.859 VND
    *   **Nhận xét:** Lượng tiền mặt và các khoản tương đương tiền cuối kỳ đã tăng mạnh, cho thấy công ty có một lượng tiền mặt dồi dào hơn để đáp ứng các nghĩa vụ ngắn hạn.

2.  **Lưu chuyển tiền thuần từ hoạt động kinh doanh:**
    *   Năm 2024: 3.825.956.276.177 VND (dòng tiền dương)
    *   Năm 2023: 3.049.557.012.386 VND (dòng tiền dương)
    *   **Nhận xét:** Công ty tiếp tục tạo ra dòng tiền dương mạnh mẽ từ các hoạt động kinh doanh cốt lõi, và con số này đã tăng lên trong năm 2024. Đây là nguồn chính để công ty thanh toán các khoản nợ ngắn hạn và tài trợ cho hoạt động.

3.  **Lưu chuyển tiền thuần trong năm:**
    *   Năm 2024: 243.467.964.400 VND (dòng tiền dương)
    *   Năm 2023: (999.370.932.730) V